In [3]:
#grouping aggregation
import pandas as pd
import numpy as np

df = pd.read_csv("./data/P87-S5-Grouping-Resources/train.csv", low_memory=False, parse_dates=["Date"])

In [4]:
df.head(2)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1


In [6]:
#single-aggregates
df.groupby("Store").mean().head(2)

,DayOfWeek,Sales,Customers,Open,Promo,SchoolHoliday
Store,,,,,,
1,3.997877,3945.704883,467.646497,0.829087,0.382166,0.204883
2,3.997877,4122.991507,486.045648,0.832272,0.382166,0.177282


In [8]:
#different aggregated for different columns
df.groupby(["Store", "DayOfWeek"]).agg({"Sales":"mean", "Customers":"count"}).head(3)

Sales  Customers
Store DayOfWeek                        
1     1          4946.119403        134
      2          4650.918519        135
      3          4454.474074        135

In [13]:
df2 = df.groupby(["Store", "DayOfWeek"]).agg({"Sales" : ["mean", "max", "min"], "Customers" : "count"})
df2.head(3)

Sales           Customers
                        mean   max min     count
Store DayOfWeek                                 
1     1          4946.119403  9528   0       134
      2          4650.918519  7959   0       135
      3          4454.474074  7821   0       135

In [14]:
df2.columns = ["SalesMean", "SalesMax", "SalesMin", "Count"]
df2.head(2)

SalesMean  SalesMax  SalesMin  Count
Store DayOfWeek                                        
1     1          4946.119403      9528         0    134
      2          4650.918519      7959         0    135

In [15]:
mc_uncert = lambda x: np.std(x) / np.sqrt(x.size)
df2 = df.groupby(["Store", "DayOfWeek"]).agg({
    "Sales": [("SalesMean", "mean"),
             ("SalesUncert", mc_uncert)],
    "Customers":"count"})
df2.head(2)

Sales             Customers
                   SalesMean SalesUncert     count
Store DayOfWeek                                   
1     1          4946.119403  139.120367       134
      2          4650.918519   94.412423       135

In [16]:
#another-way/method
def mc_uncert2(x):
    return np.std(x)/np.sqrt(x.size)

dfg = df.groupby(["Store", "DayOfWeek"])
dfg.agg(
    SalesMean = ("Sales", "mean"),
    SalesUncert = ("Sales", mc_uncert2)
).reset_index().head()

,Store,DayOfWeek,SalesMean,SalesUncert
0,1,1,4946.119403,139.120367
1,1,2,4650.918519,94.412423
2,1,3,4454.474074,97.225056
3,1,4,4094.607407,129.367657
4,1,5,4516.414815,113.495466
